## Import Packages 

In [1]:
import time
from os import path

In [2]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [3]:
import dataset
import tree as miptree
from sklearn import tree

## Optimal Classification Tree 

In [4]:
timelimit = 600
datasets = ['balance-scale', 'breast-cancer', 'car-evaluation', 'hayes-roth', 'house-votes-84', 
            'soybean-small', 'spect', 'tic-tac-toe', 'monks-1', 'monks-2', 'monks-3']
alpha = [0, 0.01, 0.1]
depth = [2, 3, 4, 5]
seeds = [37, 42, 53]

In [5]:
train_ratio = 0.5
val_ratio = 0.25
test_ratio = 0.25

In [6]:
# create or load table
res_sk = pd.DataFrame(columns=['instance', 'depth', 'seed', 'train_acc', 'val_acc', 'test_acc', 'train_time'])
if path.isfile('./res/oct.csv'):
    res_oct = pd.read_csv('./res/oct.csv')
else:
    res_oct = pd.DataFrame(columns=['instance', 'depth', 'alpha', 'seed', 
                                    'train_acc', 'val_acc', 'test_acc', 'train_time', 'gap'])
if path.isfile('./res/mfoct.csv'):
    res_mfoct = pd.read_csv('./res/mfoct.csv')
else:
    res_mfoct = pd.DataFrame(columns=['instance', 'depth', 'alpha', 'seed', 
                                      'train_acc', 'val_acc', 'test_acc', 'train_time', 'gap'])
if path.isfile('./res/boct.csv'):
    res_boct = pd.read_csv('./res/boct.csv')
else:
    res_boct = pd.DataFrame(columns=['instance', 'depth', 'seed', 
                                     'train_acc', 'val_acc', 'test_acc', 'train_time', 'gap'])
if path.isfile('./res/soct.csv'):
    res_soct = pd.read_csv('./res/soct.csv')
else:
    res_soct = pd.DataFrame(columns=['instance', 'method', 'depth', 'alpha', 'seed', 
                                     'train_acc', 'val_acc', 'test_acc', 'train_time', 'gap'])

In [7]:
for data in datasets:
    for d in depth:
        for s in seeds:
            x, y = dataset.loadData(data)
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=1-train_ratio, random_state=s)
            x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, 
                                                            test_size=test_ratio/(test_ratio+val_ratio), random_state=s)
            clf = tree.DecisionTreeClassifier(max_depth=d)
            tick = time.time()
            clf.fit(x_train, y_train)
            tock = time.time()
            train_time = tock - tick
            train_acc = accuracy_score(y_train, clf.predict(x_train))
            val_acc = accuracy_score(y_val, clf.predict(x_val))
            test_acc = accuracy_score(y_test, clf.predict(x_test))
            print(data, 'cart-d{}'.format(d), 'train acc:', train_acc, 'val acc:', val_acc)
            row = {'instance':data, 'depth':d, 'seed':s, 'train_acc':train_acc, 
                   'val_acc':val_acc, 'test_acc':test_acc, 'train_time':train_time}
            res_sk = res_sk.append(row, ignore_index=True)
            res_sk.to_csv('./res/sk.csv', index=False)

balance-scale cart-d2 train acc: 0.7275641025641025 val acc: 0.6089743589743589
balance-scale cart-d2 train acc: 0.6730769230769231 val acc: 0.6217948717948718
balance-scale cart-d2 train acc: 0.6923076923076923 val acc: 0.6217948717948718
balance-scale cart-d3 train acc: 0.782051282051282 val acc: 0.6666666666666666
balance-scale cart-d3 train acc: 0.8044871794871795 val acc: 0.6794871794871795
balance-scale cart-d3 train acc: 0.8012820512820513 val acc: 0.6923076923076923
balance-scale cart-d4 train acc: 0.8365384615384616 val acc: 0.7756410256410257
balance-scale cart-d4 train acc: 0.8301282051282052 val acc: 0.782051282051282
balance-scale cart-d4 train acc: 0.842948717948718 val acc: 0.7756410256410257
balance-scale cart-d5 train acc: 0.875 val acc: 0.782051282051282
balance-scale cart-d5 train acc: 0.8685897435897436 val acc: 0.7564102564102564
balance-scale cart-d5 train acc: 0.8814102564102564 val acc: 0.7756410256410257
breast-cancer cart-d2 train acc: 0.7753623188405797 val a

monks-3 cart-d2 train acc: 0.9711191335740073 val acc: 0.9492753623188406
monks-3 cart-d2 train acc: 0.9711191335740073 val acc: 0.9492753623188406
monks-3 cart-d2 train acc: 0.9675090252707581 val acc: 0.9492753623188406
monks-3 cart-d3 train acc: 0.9783393501805054 val acc: 0.9420289855072463
monks-3 cart-d3 train acc: 0.9711191335740073 val acc: 0.9492753623188406
monks-3 cart-d3 train acc: 0.9675090252707581 val acc: 0.9492753623188406
monks-3 cart-d4 train acc: 0.9963898916967509 val acc: 0.9637681159420289
monks-3 cart-d4 train acc: 0.9963898916967509 val acc: 0.9855072463768116
monks-3 cart-d4 train acc: 0.9927797833935018 val acc: 0.9782608695652174
monks-3 cart-d5 train acc: 0.9963898916967509 val acc: 0.9637681159420289
monks-3 cart-d5 train acc: 0.9963898916967509 val acc: 0.9855072463768116
monks-3 cart-d5 train acc: 0.9963898916967509 val acc: 0.9637681159420289


In [8]:
for data in datasets:
    for d in depth:
        for s in seeds:
            # load data
            x, y = dataset.loadData(data)
            # onehot encoding
            x_enc = dataset.oneHot(x)
            # data splition
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=1-train_ratio, random_state=s)
            x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, 
                                                            test_size=test_ratio/(test_ratio+val_ratio), random_state=s)
            x_train_enc, x_test_enc, y_train, y_test = train_test_split(x_enc, y, test_size=1-train_ratio, random_state=s)
            x_val_enc, x_test_enc, y_val, y_test = train_test_split(x_test_enc, y_test, 
                                                                    test_size=test_ratio/(test_ratio+val_ratio), random_state=s)
            
            for a in alpha:
                # oct
                row = res_oct[(res_oct['instance'] == data) & (res_oct['depth'] == d) & 
                              (res_oct['alpha'] == a) & (res_oct['seed'] == s)]
                if len(row):
                    print(data, 'oct-d{}-a{}'.format(row['depth'].values[0],row['alpha'].values[0]),
                          'train acc:', row['train_acc'].values[0], 'val acc:', row['val_acc'].values[0],
                          'gap:', row['gap'].values[0])
                else:
                    octree = miptree.optimalDecisionTreeClassifier(max_depth=d, min_samples_split=0, alpha=a,
                                                                   timelimit=timelimit, output=False)
                    tick = time.time()
                    octree.fit(x_train, y_train)
                    tock = time.time()
                    train_time = tock - tick
                    train_acc = accuracy_score(y_train, octree.predict(x_train))
                    val_acc = accuracy_score(y_val, octree.predict(x_val))
                    test_acc = accuracy_score(y_test, octree.predict(x_test))
                    row = {'instance':data, 'depth':d, 'alpha':a, 'seed':s, 'train_acc':train_acc, 'val_acc':val_acc,
                           'test_acc':test_acc, 'train_time':train_time, 'gap':octree.optgap}
                    res_oct = res_oct.append(row, ignore_index=True)
                    res_oct.to_csv('./res/oct.csv', index=False)
                    print(data, 'oct-d{}-a{}'.format(d,a), 
                          'train acc:', train_acc, 'val acc:', val_acc, 'gap:', octree.optgap)
                    #print(row)
                
                # mfoct
                row = res_mfoct[(res_mfoct['instance'] == data) & (res_mfoct['depth'] == d) & 
                                (res_mfoct['alpha'] == a) & (res_mfoct['seed'] == s)]
                if len(row):
                    print(data, 'mfoct-d{}-a{}'.format(row['depth'].values[0],row['alpha'].values[0]),
                          'train acc:', row['train_acc'].values[0], 'val acc:', row['val_acc'].values[0],
                          'gap:', row['gap'].values[0])
                else:
                    mfoct = miptree.maxFlowOptimalDecisionTreeClassifier(max_depth=d, alpha=a,
                                                                         timelimit=timelimit, output=False)
                    tick = time.time()
                    mfoct.fit(x_train_enc, y_train)
                    tock = time.time()
                    train_time = tock - tick
                    train_acc = accuracy_score(y_train, mfoct.predict(x_train_enc))
                    val_acc = accuracy_score(y_val, mfoct.predict(x_val_enc))
                    test_acc = accuracy_score(y_test, mfoct.predict(x_test_enc))
                    row = {'instance':data, 'depth':d, 'alpha':a, 'seed':s, 'train_acc':train_acc, 'val_acc':val_acc,
                           'test_acc':test_acc, 'train_time':train_time, 'gap':mfoct.optgap}
                    res_mfoct = res_mfoct.append(row, ignore_index=True)
                    res_mfoct.to_csv('./res/mfoct.csv', index=False)
                    print(data, 'mfoct-d{}-a{}'.format(d,a), 
                          'train acc:', train_acc, 'val acc:', val_acc, 'gap:', mfoct.optgap)
                    #print(row)
            
            # boct
            row = res_boct[(res_boct['instance'] == data) & (res_boct['depth'] == d) & (res_boct['seed'] == s)]
            if len(row):
                print(data, 'boct-d{}'.format(row['depth'].values[0]),
                      'train acc:', row['train_acc'].values[0], 'val acc:', row['val_acc'].values[0], 
                      'gap:', row['gap'].values[0])
            else:
                boct = miptree.binOptimalDecisionTreeClassifier(max_depth=d, min_samples_split=0,
                                                                timelimit=timelimit, output=False)
                tick = time.time()
                boct.fit(x_train, y_train)
                tock = time.time()
                train_time = tock - tick
                train_acc = accuracy_score(y_train, boct.predict(x_train))
                val_acc = accuracy_score(y_val, boct.predict(x_val))
                test_acc = accuracy_score(y_test, boct.predict(x_test))
                row = {'instance':data, 'depth':d, 'seed':s, 'train_acc':train_acc, 'val_acc':val_acc,
                       'test_acc':test_acc, 'train_time':train_time, 'gap':boct.optgap}
                res_boct = res_boct.append(row, ignore_index=True)
                res_boct.to_csv('./res/boct.csv', index=False)
                print(data, 'boct-d{}'.format(d), 'train acc:', train_acc, 'val acc:', val_acc, 'gap:', boct.optgap)
                #print(row)

KeyError: 'gap'

## Stable Optimal Classification Tree

In [9]:
seeds = [11, 23, 34, 45, 56, 67, 78, 89, 93, 5]
d = 3

In [10]:
for data in datasets:
    for s in seeds:
        
        # load data
        x, y = dataset.loadData(data)
        # onehot encoding
        x_enc = dataset.oneHot(x)
        # data splition
        x_train_enc, x_test_enc, y_train, y_test = train_test_split(x_enc, y, test_size=test_ratio, random_state=s)
        N = int(len(y_train) * train_ratio / (train_ratio + val_ratio))
        
        for a in alpha:
            
            # mfoct
            x_train_enc_s, x_val_enc_s, y_train_s, y_val_s = train_test_split(x_train_enc, y_train, 
                                                                              test_size=val_ratio/(train_ratio+val_ratio), 
                                                                              random_state=3)
            row = res_soct[(res_soct['instance'] == data) & (res_soct['method'] == 'n') & (res_soct['depth'] == d) &
                           (res_soct['alpha'] == a) & (res_soct['seed'] == s)]
            if len(row):
                print(data, 'mfoct-d{}-a{}'.format(row['depth'].values[0],row['alpha'].values[0]),
                      'method', row['method'].values[0], 'train acc:', row['train_acc'].values[0], 
                      'test acc:', row['test_acc'].values[0], row['gap'].values[0])
            else:
                mfoct = miptree.maxFlowOptimalDecisionTreeClassifier(max_depth=2, alpha=a,
                                                                     timelimit=timelimit, output=False)
                tick = time.time()
                mfoct.fit(x_train_enc_s, y_train_s)
                tock = time.time()
                train_time = tock - tick
                train_acc = accuracy_score(y_train_s, mfoct.predict(x_train_enc_s))
                val_acc = accuracy_score(y_val_s, mfoct.predict(x_val_enc_s))
                test_acc = accuracy_score(y_test, mfoct.predict(x_test_enc))
                row = {'instance':data, 'method':'n', 'depth':d, 'alpha':a, 'seed':s, 
                       'train_acc':train_acc, 'val_acc': val_acc, 'test_acc':test_acc, 
                       'train_time':train_time, 'gap':mfoct.optgap}
                res_soct = res_soct.append(row, ignore_index=True)
                res_soct.to_csv('./res/soct.csv', index=False)
                print(data, 'mfoct-d{}-a{}'.format(d,a), 'method:', 'n',
                      'train acc:', train_acc, 'test acc:', test_acc, 'gap:', mfoct.optgap)
                #print(row)

            # robust
            row = res_soct[(res_soct['instance'] == data) & (res_soct['method'] == 'rb') & (res_soct['depth'] == d) &
                           (res_soct['alpha'] == a) & (res_soct['seed'] == s)]
            if len(row):
                print(data, 'mfoct-d{}-a{}'.format(row['depth'].values[0],row['alpha'].values[0]),
                      'method', row['method'].values[0], 'train acc:', row['train_acc'].values[0], 
                      'test acc:', row['test_acc'].values[0], row['gap'].values[0])
            else:
                mfoct = miptree.maxFlowOptimalDecisionTreeClassifier(max_depth=2, alpha=a,
                                                                     timelimit=timelimit, output=False)
                tick = time.time()
                mfoct.stable_fit_robust(x_train_enc, y_train, N)
                tock = time.time()
                train_time = tock - tick
                train_acc = accuracy_score(y_train, mfoct.predict(x_train_enc))
                test_acc = accuracy_score(y_test, mfoct.predict(x_test_enc))
                row = {'instance':data, 'method':'rb', 'depth':d, 'alpha':a, 'seed':s, 
                       'train_acc':train_acc, 'test_acc':test_acc, 'train_time':train_time, 'gap':mfoct.optgap}
                res_soct = res_soct.append(row, ignore_index=True)
                res_soct.to_csv('./res/soct.csv', index=False)
                print(data, 'mfoct-d{}-a{}'.format(d,a), 'method:', 'rb',
                      'train acc:', train_acc, 'test acc:', test_acc, 'gap:', mfoct.optgap)
                #print(row)

            # cutting plane
            row = res_soct[(res_soct['instance'] == data) & (res_soct['method'] == 'cp') & (res_soct['depth'] == d) &
                           (res_soct['alpha'] == a) & (res_soct['seed'] == s)]
            if len(row):
                print(data, 'mfoct-d{}-a{}'.format(row['depth'].values[0],row['alpha'].values[0]),
                      'method', row['method'].values[0], 'train acc:', row['train_acc'].values[0], 
                      'test acc:', row['test_acc'].values[0], row['gap'].values[0])
            else:
                mfoct = miptree.maxFlowOptimalDecisionTreeClassifier(max_depth=2, alpha=a,
                                                                     timelimit=timelimit, output=False)
                tick = time.time()
                mfoct.stable_fit_CP(x_train_enc, y_train, N)
                tock = time.time()
                train_time = tock - tick
                train_acc = accuracy_score(y_train, mfoct.predict(x_train_enc))
                test_acc = accuracy_score(y_test, mfoct.predict(x_test_enc))
                row = {'instance':data, 'method':'cp', 'depth':d, 'alpha':a, 'seed':s, 
                       'train_acc':train_acc, 'test_acc':test_acc, 'train_time':train_time, 'gap':mfoct.optgap}
                res_soct = res_soct.append(row, ignore_index=True)
                res_soct.to_csv('./res/soct.csv', index=False)
                print(data, 'mfoct-d{}-a{}'.format(d,a), 'method:', 'cp',
                      'train acc:', train_acc, 'test acc:', test_acc, 'gap:', mfoct.optgap)
                #print(row)

Academic license - for non-commercial use only - expires 2021-06-13
Using license file C:\Users\Apocrypse\gurobi.lic
balance-scale mfoct-d3-a0 method: n train acc: 0.6987179487179487 test acc: 0.7197452229299363 gap: 0.0
balance-scale mfoct-d3-a0 method: rb train acc: 0.6858974358974359 test acc: 0.6687898089171974 gap: 0.0
balance-scale mfoct-d3-a0 method: cp train acc: 0.6858974358974359 test acc: 0.6687898089171974 gap: 0.0
balance-scale mfoct-d3-a0.01 method: n train acc: 0.6987179487179487 test acc: 0.7197452229299363 gap: 0.0
balance-scale mfoct-d3-a0.01 method: rb train acc: 0.6816239316239316 test acc: 0.6815286624203821 gap: 0.0
balance-scale mfoct-d3-a0.01 method: cp train acc: 0.6816239316239316 test acc: 0.6815286624203821 gap: 0.0
balance-scale mfoct-d3-a0.1 method: n train acc: 0.6923076923076923 test acc: 0.6815286624203821 gap: 0.0
balance-scale mfoct-d3-a0.1 method: rb train acc: 0.6816239316239316 test acc: 0.6815286624203821 gap: 0.0
balance-scale mfoct-d3-a0.1 metho

balance-scale mfoct-d3-a0.01 method: cp train acc: 0.6965811965811965 test acc: 0.6369426751592356 gap: 0.0
balance-scale mfoct-d3-a0.1 method: n train acc: 0.6891025641025641 test acc: 0.6560509554140127 gap: 0.0
balance-scale mfoct-d3-a0.1 method: rb train acc: 0.6901709401709402 test acc: 0.6560509554140127 gap: 0.0
balance-scale mfoct-d3-a0.1 method: cp train acc: 0.6901709401709402 test acc: 0.6560509554140127 gap: 0.0
balance-scale mfoct-d3-a0 method: n train acc: 0.6826923076923077 test acc: 0.6624203821656051 gap: 0.0
balance-scale mfoct-d3-a0 method: rb train acc: 0.6923076923076923 test acc: 0.6496815286624203 gap: 0.0
balance-scale mfoct-d3-a0 method: cp train acc: 0.6923076923076923 test acc: 0.6496815286624203 gap: 0.0
balance-scale mfoct-d3-a0.01 method: n train acc: 0.6826923076923077 test acc: 0.6624203821656051 gap: 0.0
balance-scale mfoct-d3-a0.01 method: rb train acc: 0.688034188034188 test acc: 0.6624203821656051 gap: 0.0
balance-scale mfoct-d3-a0.01 method: cp trai

breast-cancer mfoct-d3-a0 method: cp train acc: 0.8019323671497585 test acc: 0.6714285714285714 gap: 0.0
breast-cancer mfoct-d3-a0.01 method: n train acc: 0.7971014492753623 test acc: 0.6571428571428571 gap: 0.0
breast-cancer mfoct-d3-a0.01 method: rb train acc: 0.7922705314009661 test acc: 0.7142857142857143 gap: 0.0
breast-cancer mfoct-d3-a0.01 method: cp train acc: 0.7971014492753623 test acc: 0.7142857142857143 gap: 0.0
breast-cancer mfoct-d3-a0.1 method: n train acc: 0.717391304347826 test acc: 0.7142857142857143 gap: 0.0
breast-cancer mfoct-d3-a0.1 method: rb train acc: 0.7053140096618358 test acc: 0.7142857142857143 gap: 0.0
breast-cancer mfoct-d3-a0.1 method: cp train acc: 0.7053140096618358 test acc: 0.7142857142857143 gap: 0.0
breast-cancer mfoct-d3-a0 method: n train acc: 0.8115942028985508 test acc: 0.7428571428571429 gap: 0.0
breast-cancer mfoct-d3-a0 method: rb train acc: 0.7874396135265701 test acc: 0.7428571428571429 gap: 0.0
breast-cancer mfoct-d3-a0 method: cp train a

car-evaluation mfoct-d3-a0.1 method: rb train acc: 0.6959876543209876 test acc: 0.7129629629629629 gap: 0.0
car-evaluation mfoct-d3-a0.1 method: cp train acc: 0.6959876543209876 test acc: 0.7129629629629629 gap: 0.0
car-evaluation mfoct-d3-a0 method: n train acc: 0.7662037037037037 test acc: 0.7916666666666666 gap: 0.0
car-evaluation mfoct-d3-a0 method: rb train acc: 0.7731481481481481 test acc: 0.7916666666666666 gap: 0.0
car-evaluation mfoct-d3-a0 method: cp train acc: 0.7731481481481481 test acc: 0.7916666666666666 gap: 0.0
car-evaluation mfoct-d3-a0.01 method: n train acc: 0.7662037037037037 test acc: 0.7916666666666666 gap: 0.0
car-evaluation mfoct-d3-a0.01 method: rb train acc: 0.7731481481481481 test acc: 0.7916666666666666 gap: 0.0
car-evaluation mfoct-d3-a0.01 method: cp train acc: 0.7731481481481481 test acc: 0.7916666666666666 gap: 0.0
car-evaluation mfoct-d3-a0.1 method: n train acc: 0.6956018518518519 test acc: 0.7013888888888888 gap: 0.0
car-evaluation mfoct-d3-a0.1 metho

hayes-roth mfoct-d3-a0.1 method: n train acc: 0.6625 test acc: 0.6 gap: 0.0
hayes-roth mfoct-d3-a0.1 method: rb train acc: 0.6333333333333333 test acc: 0.5 gap: 0.0
hayes-roth mfoct-d3-a0.1 method: cp train acc: 0.6333333333333333 test acc: 0.5 gap: 0.0
hayes-roth mfoct-d3-a0 method: n train acc: 0.6625 test acc: 0.575 gap: 0.0
hayes-roth mfoct-d3-a0 method: rb train acc: 0.6583333333333333 test acc: 0.525 gap: 0.0
hayes-roth mfoct-d3-a0 method: cp train acc: 0.6583333333333333 test acc: 0.35 gap: 0.0
hayes-roth mfoct-d3-a0.01 method: n train acc: 0.6625 test acc: 0.525 gap: 0.0
hayes-roth mfoct-d3-a0.01 method: rb train acc: 0.6583333333333333 test acc: 0.525 gap: 0.0
hayes-roth mfoct-d3-a0.01 method: cp train acc: 0.6583333333333333 test acc: 0.45 gap: 0.0
hayes-roth mfoct-d3-a0.1 method: n train acc: 0.65 test acc: 0.475 gap: 0.0
hayes-roth mfoct-d3-a0.1 method: rb train acc: 0.6416666666666667 test acc: 0.475 gap: 0.0
hayes-roth mfoct-d3-a0.1 method: cp train acc: 0.641666666666666

...

spect mfoct-d3-a0 method: n train acc: 0.8345864661654135 test acc: 0.7761194029850746 gap: 0.0
spect mfoct-d3-a0 method: rb train acc: 0.8 test acc: 0.7761194029850746 gap: 0.0
spect mfoct-d3-a0 method: cp train acc: 0.8 test acc: 0.7761194029850746 gap: 0.0
spect mfoct-d3-a0.01 method: n train acc: 0.8345864661654135 test acc: 0.7761194029850746 gap: 0.0
spect mfoct-d3-a0.01 method: rb train acc: 0.645 test acc: 0.6119402985074627 gap: 0.0
spect mfoct-d3-a0.01 method: cp train acc: 0.8 test acc: 0.7761194029850746 gap: 0.0
spect mfoct-d3-a0.1 method: n train acc: 0.8345864661654135 test acc: 0.7761194029850746 gap: 0.0
spect mfoct-d3-a0.1 method: rb train acc: 0.8 test acc: 0.7761194029850746 gap: 0.0
spect mfoct-d3-a0.1 method: cp train acc: 0.8 test acc: 0.7761194029850746 gap: 0.0
spect mfoct-d3-a0 method: n train acc: 0.8195488721804511 test acc: 0.8059701492537313 gap: 0.0
spect mfoct-d3-a0 method: rb train acc: 0.79 test acc: 0.8059701492537313 gap: 0.0
spect mfoct-d3-a0 method

tic-tac-toe mfoct-d3-a0.01 method: cp train acc: 0.7075208913649025 test acc: 0.7 gap: 0.0
tic-tac-toe mfoct-d3-a0.1 method: n train acc: 0.6443514644351465 test acc: 0.675 gap: 0.0
tic-tac-toe mfoct-d3-a0.1 method: rb train acc: 0.6462395543175488 test acc: 0.675 gap: 0.0
tic-tac-toe mfoct-d3-a0.1 method: cp train acc: 0.6462395543175488 test acc: 0.675 gap: 0.0
tic-tac-toe mfoct-d3-a0 method: n train acc: 0.7364016736401674 test acc: 0.7166666666666667 gap: 0.0
tic-tac-toe mfoct-d3-a0 method: rb train acc: 0.7158774373259053 test acc: 0.6583333333333333 gap: 0.0
tic-tac-toe mfoct-d3-a0 method: cp train acc: 0.7158774373259053 test acc: 0.6583333333333333 gap: 0.0
tic-tac-toe mfoct-d3-a0.01 method: n train acc: 0.7364016736401674 test acc: 0.7166666666666667 gap: 0.0
tic-tac-toe mfoct-d3-a0.01 method: rb train acc: 0.7033426183844012 test acc: 0.6875 gap: 0.0
tic-tac-toe mfoct-d3-a0.01 method: cp train acc: 0.7033426183844012 test acc: 0.6875 gap: 0.0
tic-tac-toe mfoct-d3-a0.1 method:

monks-1 mfoct-d3-a0.1 method: rb train acc: 0.7529976019184652 test acc: 0.7266187050359713 gap: 0.0
monks-1 mfoct-d3-a0.1 method: cp train acc: 0.7529976019184652 test acc: 0.7266187050359713 gap: 0.0
monks-1 mfoct-d3-a0 method: n train acc: 0.7841726618705036 test acc: 0.7410071942446043 gap: 0.0
monks-1 mfoct-d3-a0 method: rb train acc: 0.7937649880095923 test acc: 0.7266187050359713 gap: 0.0
monks-1 mfoct-d3-a0 method: cp train acc: 0.7937649880095923 test acc: 0.7266187050359713 gap: 0.0
monks-1 mfoct-d3-a0.01 method: n train acc: 0.7841726618705036 test acc: 0.7410071942446043 gap: 0.0
monks-1 mfoct-d3-a0.01 method: rb train acc: 0.7937649880095923 test acc: 0.7266187050359713 gap: 0.0
monks-1 mfoct-d3-a0.01 method: cp train acc: 0.7937649880095923 test acc: 0.7266187050359713 gap: 0.0
monks-1 mfoct-d3-a0.1 method: n train acc: 0.7410071942446043 test acc: 0.7266187050359713 gap: 0.0
monks-1 mfoct-d3-a0.1 method: rb train acc: 0.7529976019184652 test acc: 0.7266187050359713 gap: 

monks-2 mfoct-d3-a0 method: n train acc: 0.6533333333333333 test acc: 0.5695364238410596 gap: 0.0
monks-2 mfoct-d3-a0 method: rb train acc: 0.6511111111111111 test acc: 0.6754966887417219 gap: 0.0
monks-2 mfoct-d3-a0 method: cp train acc: 0.6511111111111111 test acc: 0.6754966887417219 gap: 0.0
monks-2 mfoct-d3-a0.01 method: n train acc: 0.6533333333333333 test acc: 0.5695364238410596 gap: 0.0
monks-2 mfoct-d3-a0.01 method: rb train acc: 0.6511111111111111 test acc: 0.6754966887417219 gap: 0.0
monks-2 mfoct-d3-a0.01 method: cp train acc: 0.6511111111111111 test acc: 0.6754966887417219 gap: 0.0
monks-2 mfoct-d3-a0.1 method: n train acc: 0.64 test acc: 0.6754966887417219 gap: 0.0
monks-2 mfoct-d3-a0.1 method: rb train acc: 0.6511111111111111 test acc: 0.6754966887417219 gap: 0.0
monks-2 mfoct-d3-a0.1 method: cp train acc: 0.6511111111111111 test acc: 0.6754966887417219 gap: 0.0
monks-2 mfoct-d3-a0 method: n train acc: 0.6533333333333333 test acc: 0.5960264900662252 gap: 0.0
monks-2 mfoct

monks-3 mfoct-d3-a0.01 method: rb train acc: 0.9614457831325302 test acc: 0.9712230215827338 gap: 0.0
monks-3 mfoct-d3-a0.01 method: cp train acc: 0.9614457831325302 test acc: 0.9712230215827338 gap: 0.0
monks-3 mfoct-d3-a0.1 method: n train acc: 0.9565217391304348 test acc: 0.9712230215827338 gap: 0.0
monks-3 mfoct-d3-a0.1 method: rb train acc: 0.9614457831325302 test acc: 0.9712230215827338 gap: 0.0
monks-3 mfoct-d3-a0.1 method: cp train acc: 0.9614457831325302 test acc: 0.9712230215827338 gap: 0.0
monks-3 mfoct-d3-a0 method: n train acc: 0.9637681159420289 test acc: 0.9784172661870504 gap: 0.0
monks-3 mfoct-d3-a0 method: rb train acc: 0.9590361445783132 test acc: 0.9784172661870504 gap: 0.0
monks-3 mfoct-d3-a0 method: cp train acc: 0.9590361445783132 test acc: 0.9784172661870504 gap: 0.0
monks-3 mfoct-d3-a0.01 method: n train acc: 0.9637681159420289 test acc: 0.9784172661870504 gap: 0.0
monks-3 mfoct-d3-a0.01 method: rb train acc: 0.9590361445783132 test acc: 0.9784172661870504 gap: